# Welcome to my first NLP Competition! This is my first notebook attempt for the NLP with Disaster Tweets (Kaggle) Competition!

# Step 0: Kaggle Set Up

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Step 1: pip install dependencies and set global variables

In [ ]:
%pip install pandas numpy tensorflow transformers scikit-learn matplotlib

# #python.exe -m pip install --upgrade pip



### Set Global random seed to make sure we can replicate any model that we create (no randomness)

In [ ]:
import random
import tensorflow as tf
import numpy as np
import os



np.random.seed(42)
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

os.environ['TF_DETERMINISTIC_OPS'] = '1'

## Step 2: Exploring and Understanding the data

### Loading Data

In [ ]:
import pandas as pd

# Load the training data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Display the first few rows of the training data
print(train_data.head())

### Data Cleaning

I had a hard choice of whether or not to delete hashtags, but after inspecting the data, I saw that there were so many hashtags and hashtags are a crucial part of tweets so I decided that I want to keep them and then do the extra work of preprecessing them later on when I preprocess the data.

I might go and remove hashtags in the future, to see how it affects the performance. So, if you see that I decided to remove the hashtags, then now you know why!

In [ ]:
import re # Regular Expression

def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)     # Remove mentions
    text = re.sub(r'\d+', '', text)      # Remove numbers
    text = re.sub(r'[^\w\s#]', '', text)  # Remove punctuation except hashtags
    text = text.lower()                  # Convert to lowercase
    return text

train_data['clean_text'] = train_data['text'].apply(clean_text) # Apply the data cleaning process to training data
test_data['clean_text'] = test_data['text'].apply(clean_text)# Apply the data cleaning process to testing data

# Display the first few rows of the cleaned data
print(train_data[['text', 'clean_text']].head())

## Step 3: Preprocessing and Tokenization

### Tokenization and Padding/Truncation from bert-base-uncased 

- The bert-base-uncased tokenizer also has a padding/truncation feature built into it so we will use that so that we don't have to manually truncate and pad!

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_texts(texts):
    return tokenizer(
        texts.tolist(),
        max_length=64,
        padding=True,
        truncation=True,
        return_tensors='tf'
    )

train_encodings = tokenize_texts(train_data['clean_text'])
test_encodings = tokenize_texts(test_data['clean_text'])

### Analyzing the length of tokens to find the optimal maximum length for sequences

Based on the results from this, we saw that the maximum sequence length (a sequence in this context is a single tweet) was around 40, and so we will pick a multiple of 2 for better computational performance. 

So, I decided on 64!

In [ ]:
# import matplotlib.pyplot as plt

# # Tokenize the clean text without padding to get the length of each tweet
# train_data['token_length'] = train_data['clean_text'].apply(lambda x: len(tokenizer.encode(x, add_special_tokens=True)))
# test_data['token_length'] = test_data['clean_text'].apply(lambda x: len(tokenizer.encode(x, add_special_tokens=True)))

# # Plot the distribution of token lengths
# plt.hist(train_data['token_length'], bins=50, alpha=0.7, label='Train')
# plt.hist(test_data['token_length'], bins=50, alpha=0.7, label='Test')
# plt.axvline(x=128, color='r', linestyle='--', label='MAX_LEN = 128')
# plt.xlabel('Token Length')
# plt.ylabel('Frequency')
# plt.legend()
# plt.show()

# # Display some statistics
# print("Train token length statistics:")
# print(train_data['token_length'].describe())

# print("\nTest token length statistics:")
# print(test_data['token_length'].describe())


### Prepare data for training

This includes:
- Splitting data into training and validation 
- Converting data into a format that BERT can actually train on 

In [ ]:
import tensorflow as tf

train_labels = tf.convert_to_tensor(train_data['target'].values)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

# Create a validation split
val_size = int(0.2 * len(train_data))
val_dataset = train_dataset.take(val_size)
train_dataset = train_dataset.skip(val_size)

# Batch and shuffle the datasets
batch_size = 32

train_dataset = train_dataset.shuffle(10000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)


## Step 4: Building a model!!!

### Picking a Model Architecture

I am going to pick BERT but I might play around with other pretrained models later. 

This is an example of transfer learning, where I am taking a pretrained model (ex. BERT) and then training it on my specific data. No need to re-invent the wheel, especially since it will take long time to make a model from scratch and I might not get great results back since my training data size is not very good. The BERT model is training on SO MUCH data, so it's already very smart.

I am using the BERT model and by doing import TFBertForSequenceClassification, I am using the model that adds a classification head to the BERT base model. Adding a layer to a pre-trained model is a crucial part of transfer learning, and by training the model on my data, I will be setting the weights of the new head layer of the model, which is where it learns about disaster tweets and how to classify them!

In [ ]:
from transformers import TFBertForSequenceClassification, BertConfig

config = BertConfig.from_pretrained('bert-base-uncased', num_labels=2)
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

### Compiling the Model

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-8),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]
)

## Step 5: Training the Model

### Train the model


In [ ]:
history = model.fit(
    train_dataset,
    epochs=3,
    validation_data=val_dataset
)

## Step 6: Evaluating and Submitting the Results

### Predict on the test set

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings)
)).batch(32)

predictions = model.predict(test_dataset).logits
predicted_labels = tf.argmax(predictions, axis=1).numpy()

### Prepare the Submission File:

In [ ]:
# Create a submission DataFrame
submission = pd.DataFrame({'id': test_data['id'], 'target': predicted_labels})
submission.to_csv('submission_1.csv', index=False)

## Step 7: Iterating and Improving

Hyperparameter Tuning:
- Experiment with different hyperparameters such as learning rate, batch size, and the number of epochs to improve the model’s performance.

Data Augmentation:
- Consider using data augmentation techniques to increase the diversity of your training data.

Model Ensembles:
- Combine the predictions from multiple models to improve overall performance.